In [59]:
import pandas as pd
import pickle
import requests
from gensim.models import Word2Vec

In [21]:
df_user_following_list = pd.read_csv("data/user_following_list.csv")

In [25]:
df_user_following_list = df_user_following_list.fillna("")

In [27]:
users = [df_user_following_list.following_list[idx].split(",")  for idx in range(len(df_user_following_list))]

In [33]:
model = Word2Vec(users)

In [99]:
users[999]

['114624961',
 '101735242',
 '164234001',
 '138009898',
 '116046835',
 '57769396',
 '114367508',
 '149397653',
 '160075583',
 '137407035',
 '93480887',
 '130445097',
 '128035304',
 '126126797',
 '112273354',
 '137922988']

In [104]:
similarlist = model.most_similar("130445097")

for similar in similarlist:
    url_channel_info = "https://api.twitch.tv/kraken/channels/" + str(similar[0])
    response_channel_info = requests.get(url_channel_info, headers = header)
    res_json = response_channel_info.json()
    print(res_json["display_name"])

테이프2
짜누
기도
용봉탕
반짝이임
누구세요쨩
모데카이저
루나_
도현_
명훈


In [84]:
res_json

{'_id': '147607576',
 'broadcaster_language': 'ko',
 'broadcaster_type': '',
 'created_at': '2017-02-12T14:22:41Z',
 'description': '이아이디는 방송을 하지않아요www.twich.tv/sutjr1006ㄱ ㄱ 여기루',
 'display_name': '케이티폰',
 'followers': 4034,
 'game': 'League of Legends',
 'language': 'ko',
 'logo': 'https://static-cdn.jtvnw.net/user-default-pictures/cd618d3e-f14d-4960-b7cf-094231b04735-profile_image-300x300.jpg',
 'mature': False,
 'name': 'sutjr1004',
 'partner': False,
 'privacy_options_enabled': False,
 'private_video': False,
 'profile_banner': None,
 'profile_banner_background_color': None,
 'status': 'kt Pawn LOL ',
 'updated_at': '2018-04-27T05:01:01Z',
 'url': 'https://www.twitch.tv/sutjr1004',
 'video_banner': None,
 'views': 27628}